In [76]:
import bioframe as bf
from datasets import load_dataset
from gpn.data import Genome, load_table, load_dataset_from_file_or_dir
from liftover import get_lifter
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import polars as pl
from scipy.spatial.distance import cdist
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif, SelectFdr
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
import torch
from tqdm import tqdm
import yaml

from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

In [50]:
#dataset = "Height"
dataset = "IBD"
V = pd.read_parquet(f"../../results/dataset/gwas_gokcen_{dataset}_cre_matched/test.parquet")
V

,chrom,pos,ref,alt,MAF,PIP,label,consequence,tss_dist,match_group
0,1,1164939,C,A,0.062955,0.000000,False,dELS,8940,dELS_1
1,1,1468048,A,G,0.070269,1.000000,True,dELS,3716,dELS_0
2,1,1471144,G,C,0.071226,1.000000,True,pELS,620,pELS_0
3,1,6243564,C,T,0.071789,0.001052,False,pELS,614,pELS_0
4,1,6410895,T,C,0.062965,0.999993,True,dELS,9007,dELS_1
...,...,...,...,...,...,...,...,...,...,...
301,22,39457735,G,A,0.245940,1.000000,True,pELS,722,pELS_13
302,22,42286551,T,C,0.250440,0.000697,False,dELS,2553,dELS_80
303,22,46683646,A,G,0.060253,0.000000,False,dELS,9037,dELS_75
304,22,49957690,C,G,0.248880,0.999554,True,dELS,3081,dELS_80


In [97]:
#features_path = "GPN-MSA_InnerProducts"
features_path = "CADD"
#features_path = "Borzoi_L2"
features = pd.read_parquet(f"../../results/dataset/gwas_gokcen_{dataset}_cre_matched/features/{features_path}.parquet")
features

,GC,CpG,motifECount,motifEHIPos,motifEScoreChng,minDistTSS,minDistTSE,priPhCons,mamPhCons,verPhCons,...,RegSeq4,RegSeq5,RegSeq6,RegSeq7,ZooPriPhyloP,ZooVerPhyloP,ZooRoCC,Roulette-MR,Roulette-AR,RawScore
0,0.728,0.120,0.0,0.0,0.000,2166,2260,0.004,0.004,0.000,...,-0.002422,-0.012463,-0.006394,0.007449,0.005,0.254,0.0,0.030,0.105,0.339867
1,0.662,0.067,0.0,0.0,0.000,5565,616,0.020,0.000,0.000,...,-0.000089,0.000030,-0.000210,0.003044,0.001,-1.018,0.0,0.083,0.105,-0.226352
2,0.517,0.067,0.0,0.0,0.000,622,981,0.015,0.015,0.015,...,-0.059325,0.005445,0.021399,0.008557,0.008,-1.385,0.0,0.041,0.041,0.070621
3,0.636,0.053,0.0,0.0,0.000,616,2015,0.058,1.000,1.000,...,0.000625,-0.000423,0.002826,0.004220,0.320,5.909,25.0,0.128,0.128,2.463754
4,0.517,0.053,0.0,0.0,0.000,5737,1524,0.005,0.008,0.008,...,0.000364,0.001244,0.001680,-0.011233,0.005,-1.125,0.0,0.105,0.105,-0.017440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,0.821,0.267,0.0,0.0,0.000,723,34460,0.100,0.919,0.565,...,-0.007857,-0.022018,-0.005359,0.013065,0.012,0.828,0.0,0.117,0.105,0.877372
302,0.404,0.027,0.0,0.0,0.000,31689,71704,0.006,0.000,0.000,...,-0.004903,-0.004016,-0.008280,-0.021105,0.027,0.731,0.0,0.051,0.105,0.678291
303,0.278,0.013,0.0,0.0,0.000,54607,765,0.087,0.086,0.084,...,0.009435,0.021430,0.037646,-0.049947,0.003,0.708,0.0,0.083,0.105,0.367680
304,0.649,0.000,0.0,0.0,0.000,3079,3957,0.151,0.000,0.000,...,-0.000046,0.000003,-0.000284,0.002480,0.016,0.480,0.0,0.041,0.105,-0.228360


In [98]:
#model = SelectKBest(score_func=f_classif, k=10)
model = SelectFdr(score_func=f_classif, alpha=0.05)
model.fit(features, V["label"])
model

ValueError: Input X contains NaN.
SelectFdr does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [57]:
np.argsort(model.scores_)[::-1][:10]

array([688, 147, 263, 137, 108, 580, 343, 221, 526, 552])

In [58]:
sum(model.get_support())

0

In [106]:
X = features
y = V.label
mask_train = V.chrom.isin([str(i) for i in range(1, 23, 2)] + ["X"])
X_train = X[mask_train]
y_train = y[mask_train]
X_test = X[~mask_train]
y_test = y[~mask_train]
balanced = True

clf = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('scaler', RobustScaler()),
    #('feature_selection', SelectKBest(score_func=f_classif, k=30)),
    #('feature_selection', SelectFdr(score_func=f_classif, alpha=0.05)),
    #('feature_selection', SelectFromModel(LogisticRegression(C=1e-1, penalty="l1", random_state=42, solver="saga"))),
    ('linear', LogisticRegressionCV(
        class_weight="balanced",
        scoring="roc_auc" if balanced else "average_precision",
        Cs=np.logspace(-10, 10, 41),
        cv=3,
        random_state=42,
        n_jobs=-1,
    ))
    #(
    #    'random_forest',
    #    RandomForestClassifier(
    #        class_weight="balanced",
    #        n_estimators=1000,
    #        random_state=42,
    #        n_jobs=-1,
    #    )
    #),
])
clf.fit(X_train, y_train)
pred = clf.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, pred)

/accounts/projects/yss/gbenegas/.local/lib/python3.11/site-packages/sklearn/impute/_base.py:558: UserWarning: Skipping features without any observed values: ['MMSp_acceptorIntron' 'MMSp_acceptor']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/accounts/projects/yss/gbenegas/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/accounts/projects/yss/gbenegas/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP

0.5800303592511384